In [ ]:
import tensorflow as tf
import numpy as np
import os


In [ ]:
#in interactive mode, there is no __file__ defined
#os.getcwd() is a workaround
modelpath=os.path.dirname(__file__) if('__file__') in dir() else os.getcwd()

In [ ]:
def read_csv(batch_size,file_name,record_defaults):
    filename_queue = tf.train.string_input_producer ( 
        [os.path.join(modelpath,file_name)])
    reader = tf.TextLineReader(skip_header_lines=1)
    key,value=reader.read(filename_queue)
    #decode-csv will convert a Tensor from type string (the text line) 
    #in a tuple of tensor columns with the specified defaults, 
    #which also sets the data type for each column
    decoded=tf.decode_csv(value,record_defaults=record_defaults)
    
    #batch actually reads the file and loads "batch_size" rows in a single tensor
    
    return tf.train.shuffle_batch(decoded,batch_size=batch_size,
                                 capacity=batch_size*50, 
                                 min_after_dequeue=batch_size)
    

In [ ]:
#demo of logistic to answer Yes/No question
#initialize variables/model parameters
W=tf.Variable(tf.zeros([4,3]),dtype=tf.float32,name="weights")
b = tf.Variable(tf.zeros([3],name="bias",dtype=tf.float32))

In [ ]:
def inputs():
    batch_size=100
    sepal_length,sepal_width,petal_length,petal_width,label = \
     read_csv(batch_size,'irisData/iris.data', [[0.0], [0.0], [0.0], [0.0], [""]])
    #convert class names to a 0 based class index
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.pack([
        tf.equal(label, ["Iris-setosa"]),
        tf.equal(label, ["Iris-versicolor"]),
        tf.equal(label, ["Iris-virginica"])
    ])), 0))
    
    #pack all features in a single matrix
    #transpose to a matrix with one example per row
    #and each feature dimension per column
    features = tf.transpose(tf.stack(
    [sepal_length,sepal_width,petal_length,petal_width]
    ))
    
    return features, label_number

In [ ]:
def combine_inputs(X):
    return tf.matmul(X,W)+b
#define the training loop operations
def inference(X):
    #compute the inference model over data X and return the result
    #return tf.sigmoid(combine_inputs(X))
    return tf.nn.softmax(combine_inputs(X))
def loss(X,Y):
    #compute loss over training data X and expected output Y
    Y_predicted = inference(X)
    
    #applying cross entropy reduction
    err=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=combine_inputs(X),labels=Y))
    return err

def train(total_loss):
    #train/adjust model parameters according to computed total loss
    learning_rate = 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
    return optimizer

In [1]:
def evaluate(sess,X,Y):
    #evaluate the resulting trained model
    #softmax output a column matrix, axis=1
    pre=inference(X)
    predicted = tf.cast( tf.argmax(  inference(X) ,1),tf.int32 )
    result=sess.run(tf.reduce_mean(tf.cast (tf.equal(predicted,Y),tf.float32)))
    print("softmax dim:{0}".format(tf.shape(pre)))
    return

In [ ]:
#Create a saver
saver = tf.train.Saver()

In [ ]:
#Launch the graph in a session, setup boilerplate
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X,Y = inputs()
    total_loss = loss(X,Y)
    train_op = train(total_loss)
    coord = tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    
    initial_step=0
    
    #verify if we don't have a checkpoint saved already
    ckpt = tf.train.get_checkpoint_state(modelpath)
    
    if ckpt and ckpt.model_checkpoint_path:
        #Restores from checkpoints
        saver.restore(sess,ckpt.model_checkpoint_path)
        initial_step = int(ckpt.model_checkpoint_path.rsplit('-',1)[1])
    
    #actual training loop
    training_steps=2000
    
    for step in range(initial_step,training_steps):
        sess.run([train_op])
        if step%1000 ==0:
            saver.save(sess,"my-model",global_step=step)
        #for debugging and learning purposes, see how the loss gets decremented through training steps
        if step%10 ==0:
            print("loss: {0}".format(sess.run([total_loss])))
            evaluate(sess,X,Y)
            
    coord.request_stop()
    coord.join(threads)
    saver.save(sess,"my-model",global_step=training_steps)
    sess.close()